In [1]:
import gc
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import plotly.express as px
import plotly.io as pio
import random
import time
import torch
import torch.nn as nn
import torch.nn.functional as F


from glob import glob
from torch.utils.data import DataLoader, Dataset, Subset
from tqdm import tqdm

!mkdir output
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
pio.renderers.default = 'iframe'

�T�u�f�B���N�g���܂��̓t�@�C�� output �͊��ɑ��݂��܂��B


In [2]:
class config:
    AMP = False
    BATCH_SIZE_TRAIN = 32
    BATCH_SIZE_VALID = 32
    DEBUG = False
    EPOCHS = 30
    FOLDS = 5
    GRADIENT_ACCUMULATION_STEPS = 1
    LEARNING_RATE = 1e-3
    MAX_GRAD_NORM = 1e7
    NUM_WORKERS = 0 # multiprocessing.cpu_count()
    PRINT_FREQ = 20
    SEED = 20
    TRAIN_FULL_DATA = False
    WEIGHT_DECAY = 0.01


class paths:
    OUTPUT_DIR = "C:\\Users\\konno\\SynologyDrive\\datasciense\\projects_foler\\1_kaggle\\CMI\\cmi-detect-behavior-with-sensor-data\\output"
    TEST_CSV = "C:\\Users\\konno\\SynologyDrive\\datasciense\\projects_foler\\1_kaggle\\CMI\\cmi-detect-behavior-with-sensor-data\\test.csv"
    TEST_DEMOGRAPHICS = "C:\\Users\\konno\\SynologyDrive\\datasciense\\projects_foler\\1_kaggle\\CMI\\cmi-detect-behavior-with-sensor-data\\test_demographics.csv"
    TRAIN_CSV = "C:\\Users\\konno\\SynologyDrive\\datasciense\\projects_foler\\1_kaggle\\CMI\\cmi-detect-behavior-with-sensor-data\\train.csv"
    TRAIN_DEMOGRAPHICS = "C:\\Users\\konno\\SynologyDrive\\datasciense\\projects_foler\\1_kaggle\\CMI\\cmi-detect-behavior-with-sensor-data\\train_demographics.csv"

In [3]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s: float):
    "Convert to minutes."
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since: float, percent: float):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))


def get_logger(filename=paths.OUTPUT_DIR):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger


def format_for_scoring(df_preds: pd.DataFrame) ->tuple[pd.DataFrame, pd.DataFrame]: 
    solution = df_preds[["sequence_id", "y_true"]].copy()
    solution.columns = ["id", "gesture"]
    solution["gesture"] = solution["gesture"].map(num_to_label)

    submission = df_preds[["sequence_id", "y_pred"]].copy()
    submission.columns = ["id", "gesture"]
    submission["gesture"] = submission["gesture"].map(num_to_label)
    
    return solution, submission
    

def seed_everything(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed) 
    

def sep():
    print("—"*100)


label_to_num = {
    'Above ear - pull hair': 0,  # < ------- TARGETS START
    'Cheek - pinch skin': 1,
    'Eyebrow - pull hair': 2,
    'Eyelash - pull hair': 3,
    'Forehead - pull hairline': 4,
    'Forehead - scratch': 5,
    'Neck - pinch skin': 6,
    'Neck - scratch': 7,  # < ------- TARGETS END
    'Drink from bottle/cup': 8,  # < ------- NON-TARGETS START
    'Feel around in tray and pull out an object': 8,
    'Glasses on/off': 8,
    'Pinch knee/leg skin': 8,
    'Pull air toward your face': 8,
    'Scratch knee/leg skin': 8,
    'Text on phone': 8,
    'Wave hello': 8,
    'Write name in air': 8,
    'Write name on leg': 8  # < ------- NON-TARGETS END
}
type_to_num = {"Target": 1, "Non-Target":0}
num_to_label = {v: k for k, v in label_to_num.items()}
num_to_type = {v: k for k, v in type_to_num.items()}
LOGGER = get_logger()
seed_everything(config.SEED)

In [4]:
df_train = pd.read_csv(paths.TRAIN_CSV)
df_test = pd.read_csv(paths.TEST_CSV)
df_train_demographics = pd.read_csv(paths.TRAIN_DEMOGRAPHICS)
df_test_demographics = pd.read_csv(paths.TEST_DEMOGRAPHICS)

print(f"Train dataframe shape: {df_train.shape}"), sep()
print(f"Tesat dataframe shape: {df_test.shape}"), sep()
print(f"Train demographics dataframe shape: {df_train_demographics.shape}"), sep()
print(f"Test demographics dataframe shape: {df_test_demographics.shape}")

display(df_train.head())
display(df_train_demographics.head())

Train dataframe shape: (574945, 341)
————————————————————————————————————————————————————————————————————————————————————————————————————
Tesat dataframe shape: (107, 336)
————————————————————————————————————————————————————————————————————————————————————————————————————
Train demographics dataframe shape: (81, 8)
————————————————————————————————————————————————————————————————————————————————————————————————————
Test demographics dataframe shape: (2, 8)


,row_id,sequence_type,sequence_id,sequence_counter,subject,orientation,behavior,phase,gesture,acc_x,...,tof_5_v54,tof_5_v55,tof_5_v56,tof_5_v57,tof_5_v58,tof_5_v59,tof_5_v60,tof_5_v61,tof_5_v62,tof_5_v63
0,SEQ_000007_000000,Target,SEQ_000007,0,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.683594,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,SEQ_000007_000001,Target,SEQ_000007,1,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.949219,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,SEQ_000007_000002,Target,SEQ_000007,2,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.722656,...,-1.0,-1.0,112.0,119.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,SEQ_000007_000003,Target,SEQ_000007,3,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.601562,...,-1.0,-1.0,101.0,111.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,SEQ_000007_000004,Target,SEQ_000007,4,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.566406,...,-1.0,-1.0,101.0,109.0,125.0,-1.0,-1.0,-1.0,-1.0,-1.0


,subject,adult_child,age,sex,handedness,height_cm,shoulder_to_wrist_cm,elbow_to_wrist_cm
0,SUBJ_000206,1,41,1,1,172.0,50,25.0
1,SUBJ_001430,0,11,0,1,167.0,51,27.0
2,SUBJ_002923,1,28,1,0,164.0,54,26.0
3,SUBJ_003328,1,33,1,1,171.0,52,25.0
4,SUBJ_004117,0,15,0,1,184.0,54,28.0


### Data Pre-Processing

In [5]:
df_train["target"] = df_train["gesture"].map(label_to_num)
df_train.drop(columns=["behavior", "orientation"], inplace=True)  # TODO: use LabelEncoder
df_train["sequence_type"] = df_train["sequence_type"].map(type_to_num)
df_train.reset_index(inplace=True)

In [6]:
df_train.head(2)

,index,row_id,sequence_type,sequence_id,sequence_counter,subject,phase,gesture,acc_x,acc_y,...,tof_5_v55,tof_5_v56,tof_5_v57,tof_5_v58,tof_5_v59,tof_5_v60,tof_5_v61,tof_5_v62,tof_5_v63,target
0,0,SEQ_000007_000000,1,SEQ_000007,0,SUBJ_059520,Transition,Cheek - pinch skin,6.683594,6.214844,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1
1,1,SEQ_000007_000001,1,SEQ_000007,1,SUBJ_059520,Transition,Cheek - pinch skin,6.949219,6.214844,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1


### Validation

In [19]:
# !pip install scikit-learn

In [7]:
from sklearn.model_selection import KFold, StratifiedGroupKFold


sgkf = StratifiedGroupKFold(n_splits=config.FOLDS)
for fold, (train_index, valid_index) in enumerate(sgkf.split(df_train, df_train.target, df_train.subject)):
    df_train.loc[valid_index, "fold"] = int(fold)
    
display(df_train.groupby('fold').size()), sep()
display(df_train.head())

fold
0.0    115981
1.0    113987
2.0    117791
3.0    112982
4.0    114204
dtype: int64

————————————————————————————————————————————————————————————————————————————————————————————————————


,index,row_id,sequence_type,sequence_id,sequence_counter,subject,phase,gesture,acc_x,acc_y,...,tof_5_v56,tof_5_v57,tof_5_v58,tof_5_v59,tof_5_v60,tof_5_v61,tof_5_v62,tof_5_v63,target,fold
0,0,SEQ_000007_000000,1,SEQ_000007,0,SUBJ_059520,Transition,Cheek - pinch skin,6.683594,6.214844,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1,2.0
1,1,SEQ_000007_000001,1,SEQ_000007,1,SUBJ_059520,Transition,Cheek - pinch skin,6.949219,6.214844,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1,2.0
2,2,SEQ_000007_000002,1,SEQ_000007,2,SUBJ_059520,Transition,Cheek - pinch skin,5.722656,5.410156,...,112.0,119.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1,2.0
3,3,SEQ_000007_000003,1,SEQ_000007,3,SUBJ_059520,Transition,Cheek - pinch skin,6.601562,3.531250,...,101.0,111.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1,2.0
4,4,SEQ_000007_000004,1,SEQ_000007,4,SUBJ_059520,Transition,Cheek - pinch skin,5.566406,0.277344,...,101.0,109.0,125.0,-1.0,-1.0,-1.0,-1.0,-1.0,1,2.0


In [8]:
def min_max_scale(arr: np.ndarray) -> np.ndarray:
    min_vals = np.nanmin(arr, axis=0)
    max_vals = np.nanmax(arr, axis=0)
    ranges = np.where(max_vals - min_vals == 0, 1, max_vals - min_vals)
    scaled = (arr - min_vals) / ranges
    return scaled


def standard_scale(arr: np.ndarray) -> np.ndarray:
    means = np.nanmean(arr, axis=0)
    stds = np.nanstd(arr, axis=0)
    stds = np.where(stds == 0, 1, stds)  # Prevent division by zero for constant columns
    scaled = (arr - means) / stds
    return scaled


def pad_or_truncate(
    arr: np.ndarray,
    max_length: int = 200,
    pad_value: int = 0,
    mode: str = "random"  # "regular" or "random"
) -> np.ndarray:
    L, D = arr.shape

    if L > max_length:
        return arr[:max_length, :]

    elif L < max_length:
        if mode == "regular":
            padding = np.full((max_length - L, D), pad_value)
            return np.vstack((arr, padding))
        
        elif mode == "random":
            total_padding = max_length - L
            pad_start = np.random.randint(0, total_padding + 1)
            pad_end = total_padding - pad_start

            start_padding = np.full((pad_start, D), pad_value)
            end_padding = np.full((pad_end, D), pad_value)

            return np.vstack((start_padding, arr, end_padding))
        
        else:
            raise ValueError(f"Unknown mode: {mode}. Use 'regular' or 'random'.")

    else:
        return arr

imu_cols = ["acc_x", "acc_y", "acc_z", "rot_w", "rot_x", "rot_y", "rot_z"]
all_X, all_y, all_y_hard = [], [], []

for sequence_id in tqdm(df_train.sequence_id.unique()):
    ds = df_train[df_train["sequence_id"] == sequence_id]
    X = ds[imu_cols].values
    X = pad_or_truncate(X)
    y = ds.target.values[0]
    y_hard = ds.sequence_type.values[0]
    X = np.concatenate((standard_scale(X[:, 0:3]), X[:, 3:]), axis=1)
    X = np.where(np.isnan(X), 0.0, X)  # fill NaNs
    all_X.append(X)
    all_y.append(y)
    all_y_hard.append(y_hard)

all_X = np.array(all_X)
all_y = np.array(all_y)
all_y_hard = np.array(all_y_hard)

100%|██████████| 8151/8151 [03:00<00:00, 45.14it/s]


In [10]:
class CustomDataset(Dataset):
    def __init__(
        self, config, df: pd.DataFrame, X: np.ndarray, y: np.ndarray, y_hard: np.ndarray
    ): 
        
        self.config = config
        self.df = df
        self.X = X
        self.y = y
        self.y_hard = y_hard
        self.indexes = self.df.sequence_id.unique()
        
    def __len__(self):
        """
        Length of dataset.
        """
        return len(self.indexes)
        
    def __getitem__(self, index):
        """
        Get one item.
        """
        sequence_id = self.indexes[index]
        X = self.X[index]
        y = self.y[index]
        y_hard = self.y_hard[index]
        output = {
            "X": torch.tensor(X, dtype=torch.float32),
            "y": torch.tensor(y, dtype=torch.long),
            "y_hard": torch.tensor(y_hard, dtype=torch.float32),
            "sequence_id": sequence_id
        }
        return output     

In [11]:
train_dataset = CustomDataset(config, df_train, all_X, all_y, all_y_hard)
train_loader = DataLoader(
    train_dataset,
    batch_size=config.BATCH_SIZE_TRAIN,
    shuffle=False,
    num_workers=config.NUM_WORKERS, pin_memory=True, drop_last=True
)
idx = np.random.choice(range(0, len(train_dataset)))
cols = imu_cols
X = train_dataset[idx]["X"]
y = train_dataset[idx]["y"].item()
y_hard = train_dataset[idx]["y_hard"].item()
target = num_to_label[y]
sequence_id = train_dataset[idx]["sequence_id"]
N = X.shape[0]
df = pd.DataFrame(X, columns=cols)
df['step'] = range(N)
df_melted = df.melt(id_vars='step', var_name='sequence', value_name='value')

fig = px.line(
    df_melted,
    x='step',
    y='value',
    color='sequence',
    title=f"Sequences for {sequence_id} | Target: {target}",
    template='plotly_dark',
    width=800,
    height=600
)

fig.show(renderer='browser')